In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from thefuzz import fuzz
from thefuzz import process

# Clean up data from details page dataset

In [3]:
df_d=pd.read_json('bizbuysell.detail.lines.json',lines=True)
df_d['id']=df_d.s_id
df_d['src']='details'
df_d['url']=df_d['s_url']
df_d['title']=df_d['s_name']
df_d['desc']=df_d['s_desc']
df_d['local']=df_d['s_local']
df_d['region']=df_d['s_region']
df_d['location']=df_d['p_location']
df_d['title_loc']=df_d['title'].str.extract(r"in (.+) - BizBuySell")
df_d['categories']=df_d['s_breadcrumbs']
df_d['details']=df_d['p_details_text']
df_d['financials']=df_d['p_financials_text']
df_d['price']=df_d['s_price']
df_d['similar']=df_d['s_similar']

In [4]:
df_dd=df_d[['id','src','url','title',
            'desc','local','region','location','title_loc','categories','similar',
            'details','financials','price']]

In [5]:
df_dd.iloc[0]

id                                                    2067319.0
src                                                     details
url           https://www.bizbuysell.com/Business-Real-Estat...
title         Popular Marina in Central New Jersey in Middle...
desc          It is fully owned by a man and his wife since ...
local                                                 Middlesex
region                                               New Jersey
location                                   Middlesex County, NJ
title_loc     Central New Jersey in Middlesex County, New Je...
categories    [Real Estate For Sale, New Jersey, Marinas and...
similar                             [1864705, 2071534, 2057087]
details       Location:\nMiddlesex County, NJ\nType:\nOther ...
financials    Asking Price:\n$2,500,000\n                   ...
price                                                 2500000.0
Name: 0, dtype: object

# Clean up data from listings dataset

In [6]:
def proc_fin(row):
    result=''
    if pd.isna(row['asking_price'])==False:
        result+=f'Asking Price:\n{row["asking_price"]}\n'
    if pd.isna(row['cash_flow'])==False:
        result+=f'{row["cash_flow"]}\n'        
    return result

In [7]:
df_l=pd.read_json('bizbuysell.list.lines.json',lines=True)
df_l['id']=df_l.s_id
df_l['src']='listings'
df_l['url']='https://www.bizbuysell.com'+df_l['s_url']
df_l['title']=df_l['s_name']
df_l['desc']=df_l['s_desc']
df_l['local']=df_l['s_local']
df_l['region']=df_l['s_region']
df_l['location']=df_l['loc']
df_l['title_loc']=df_l['title'].str.extract(r"in (.+) - BizBuySell")
df_l['categories']=df_l['s_breadcrumbs']
df_l['details']=''
df_l['financials']=df_l.apply(proc_fin,axis=1)
df_l['price']=df_d['s_price'].replace('$','').replace(',','')
df_l['similar']=[list() for x in range(len(df_l.index))]

In [8]:
df_ll=df_l[['id','src','url','title',
            'desc','local','region','location','title_loc','categories', 'similar',
            'details','financials','price']]

In [9]:
df_ll.iloc[0]

id                                                      1972353
src                                                    listings
url           https://www.bizbuysell.com/Business-Opportunit...
title                    Turnkey Jewelry Store in the Caribbean
desc          Almost 33 years ago, the founders of The Natur...
local                                                      None
region                                            Christiansted
location                                          Christiansted
title_loc                                                   NaN
categories    [Businesses For Sale, Travel Businesses For Sa...
similar                                                      []
details                                                        
financials                          Asking Price:\n$2,000,000\n
price                                                 2500000.0
Name: 0, dtype: object

# Combine datasets

In [10]:
df=pd.concat([df_dd,df_ll])

In [11]:
df.shape

(302163, 14)

# Enrich

In [12]:
df['franchise']=df.title.str.contains("franchise", case=False)
df['lease']=df.title.str.contains("lease", case=False)
df['auction']=df.title.str.contains("auction", case= False)
df['rent']=df.title.str.contains("rent", case= False)

In [13]:
df['location_2']=df['location'].fillna(value='')
df['nationwide']=df.location_2.str.contains("Available Nationwide", case=False)
df['nationwide'].fillna(value=False, inplace=True)
df['multiple_locations']=df.location_2.str.contains("Available in Multiple Locations", case=False)
df['multiple_locations'].fillna(value=False, inplace=True)
df['relocatable']=df.location_2.str.contains("Relocatable", case=False)
df['relocatable'].fillna(value=False, inplace=True)

In [14]:
def proc_loc(row):
    result=None
    if pd.isna(row['location'])==False:
        l=str(row['location']).lower().strip()
        result= l
    elif pd.isna(row['local'])==False and pd.isna(row['region'])==False:
        l=str(row['local']).lower().strip()
        r=str(row['region']).lower().strip()
        result= f"{l}, {r}"
    elif pd.isna(row['title_loc'])==False:
        l=str(row['title_loc']).lower().strip()
        result= l
    if result !=None:
        for r in ['(relocatable)','available in']:
            result=result.replace(r,'')
        result=result.strip()
    return result
df['proc_loc'] = df.apply(proc_loc, axis=1)

# Get rid of franchise, lease, auction, rent

In [15]:
#df=df.loc[(df.franchise==False)&(df.lease==False)&(df.acution==False)&(df.rent==False)]
df=df.loc[(df.lease==False)&(df.auction==False)&(df.rent==False)]

In [16]:
df.shape

(294623, 23)

# Get rid of nationwide and multiple location listings

In [17]:
df=df.loc[(df.nationwide==False)&(df.multiple_locations==False)]

In [18]:
df.shape

(287470, 23)

# Get rid of duplicates by id

In [19]:
df=df.drop_duplicates(['id'])

In [20]:
df.shape

(38104, 23)

# Clean up null data

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38104 entries, 0 to 263471
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  38104 non-null  float64
 1   src                 38104 non-null  object 
 2   url                 38104 non-null  object 
 3   title               38104 non-null  object 
 4   desc                38104 non-null  object 
 5   local               33717 non-null  object 
 6   region              34014 non-null  object 
 7   location            37980 non-null  object 
 8   title_loc           37062 non-null  object 
 9   categories          38104 non-null  object 
 10  similar             38104 non-null  object 
 11  details             37952 non-null  object 
 12  financials          38104 non-null  object 
 13  price               37775 non-null  float64
 14  franchise           38104 non-null  object 
 15  lease               38104 non-null  object 
 16  auction 

In [22]:
df=df.loc[(df.price.isna()==False) & (df.proc_loc.isna()==False)]
df['details'].fillna(value='', inplace=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37773 entries, 0 to 37743
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  37773 non-null  float64
 1   src                 37773 non-null  object 
 2   url                 37773 non-null  object 
 3   title               37773 non-null  object 
 4   desc                37773 non-null  object 
 5   local               33439 non-null  object 
 6   region              33690 non-null  object 
 7   location            37713 non-null  object 
 8   title_loc           37060 non-null  object 
 9   categories          37773 non-null  object 
 10  similar             37773 non-null  object 
 11  details             37773 non-null  object 
 12  financials          37773 non-null  object 
 13  price               37773 non-null  float64
 14  franchise           37773 non-null  object 
 15  lease               37773 non-null  object 
 16  auction  

In [24]:
#fixme
df=df[:500]

## Clean up title
get rid of locaiton and 'Bizbuysell'

In [25]:
#bbs='- BizBuySell'

## Clean up location
turn into: city, state, county

## Clean up cateogories
remove repetative data, turn into binary columns

In [26]:
cats=[]
cat_list=[]
for index, row in df.iterrows():
    r=dict(id=row['id'])
    for c in row['categories']:
        r[f'category {c.lower().strip()}']=1.0
        cat_list.append(c.lower().strip())
    cats.append(r)

In [27]:
cats_df=pd.DataFrame.from_records(cats)
cats_df=cats_df.fillna(0.0)
cats_df

,id,category real estate for sale,category new jersey,category marinas and fishing,category middlesex county,category other,category mays landing,category florida,category pinellas county,category maryland,...,category freeburg,category car dealerships,category fostoria,category new britain,category lakewood,category steubenville,category lyerly,category west covina,category glens falls,category raymond
0,2067319.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1990890.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2039720.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1576680.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2087638.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2083285.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,2027927.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
497,2066645.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
498,2044121.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [28]:
#cat_list

In [29]:
# titles=[]
# for index, row in df.iterrows():
#     r=dict(id=row['id'])
#     t= row['title']
#     t=t.replace('- BizBuySell','')
#     t=t.lower()
#     for c in cat_list:
#         t=t.replace(c,'')
#     t=t.replace('in ,','')
#     r['title']=t.strip()
#     titles.append(r)
# print(titles)

In [30]:
titles_df=df[['id','title']]
titles_df.replace('- BizBuySell','',inplace=True)
# titles_df=titles_df.fillna(0.0)
titles_df

,id,title
0,2067319.0,Popular Marina in Central New Jersey in Middle...
1,1990890.0,"High Exposure in Mays Landing, New Jersey - Bi..."
3,2039720.0,"Duplex, Short term or Long term in Pinellas Co..."
4,1576680.0,"Major Price Reduction Rest/Tavern/ 13,000sq. f..."
5,2087638.0,"Business & Real Estate in Casco, Michigan - Bi..."
...,...,...
514,2083285.0,Coin Laundry with Building for Sale in Chicago...
515,2027927.0,Under Contract! Gas Station with Property for ...
516,2066645.0,High Volume Independent Gas Station & Property...
517,2044121.0,"Steve's Place in Glens Falls, New York - BizBu..."


## Clean up similar
turn into binary columns, will use PCA for feature reduction

In [31]:
sim=[]
for index, row in df.iterrows():
    r=dict(id=row['id'])
    for c in row['similar']:
        r[f'similar {c.lower()}']=1.0
    sim.append(r)

In [32]:
sim_df=pd.DataFrame.from_records(sim)
sim_df=sim_df.fillna(0.0)
sim_df

,id,similar 1864705,similar 2071534,similar 2057087,similar 2075819,similar 2035549,similar 2067510,similar 2054271,similar 2082461,similar 2050777,...,similar 2050024,similar 2078466,similar 2002773,similar 2052861,similar 2005086,similar 1491940,similar 2059484,similar 2033410,similar 2067977,similar 2032966
0,2067319.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1990890.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2039720.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1576680.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2087638.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2083285.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,2027927.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,2066645.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,2044121.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Clean up details 

get rid of repetative data

In [33]:
# det=[]
# for index, row in df.iterrows():
#     r=dict(id=row['id'])
#     for i in row['details'].split('\n'):
#         #print(i)
#         k=''
#         if i[:-1] == ':':
#             k=i[:-2].lower().strip()
# #         elif ':' in i:
# #             k,v=i.split(':',1)
# #             r[k.lower().strip()]=v.lower().strip()
#         else:
#             #assert(len(k))
#             r[k]=i.lower().strip()
#             k=''
#     det.append(r)      
# det

## Finanicals

Turn into columns

# Final dataframe

In [34]:
# df=df[['id','url','title',
#             'desc','location','categories','similar',
#             'details','financials','price']]

In [35]:
#df.head()

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 518
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  500 non-null    float64
 1   src                 500 non-null    object 
 2   url                 500 non-null    object 
 3   title               500 non-null    object 
 4   desc                500 non-null    object 
 5   local               485 non-null    object 
 6   region              485 non-null    object 
 7   location            500 non-null    object 
 8   title_loc           499 non-null    object 
 9   categories          500 non-null    object 
 10  similar             500 non-null    object 
 11  details             500 non-null    object 
 12  financials          500 non-null    object 
 13  price               500 non-null    float64
 14  franchise           500 non-null    object 
 15  lease               500 non-null    object 
 16  auction      

In [37]:
#df.to_excel('bizbuysell.single.large.ds.xlsx')

In [38]:
df_2=df[['id','price']].merge(titles_df, on='id')
df_2=df_2.merge(df[['id','desc']], on='id')
df_2=df_2.merge(cats_df, on='id')
df_2=df_2.merge(sim_df, on='id')

In [39]:
df_2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Columns: 815 entries, id to similar 2032966
dtypes: float64(813), object(2)
memory usage: 3.1+ MB


In [40]:
df_2.to_excel('bizbuysell.single.large.ds.2.xlsx')

In [41]:
#df_2.to_excel('bizbuysell.single.large.ds.2.xlsx')